In [1]:
from PIL import Image
import numpy as np

In [2]:
import gzip
import pickle
with gzip.open("../Week02/mnist.pkl.gz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [3]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

In [4]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])
print(train_y)

[5 0 4 ..., 8 4 8]


## 開始 Tensorflow

In [5]:
import tensorflow as tf
from tfdot import tfdot

# Multilayer Convolutional Network

In [6]:
from tensorflow.python.platform import gfile
with gfile.FastGFile("mnist_simple.pb",'rb') as f:
    graph_def = tf.GraphDef()
    x = f.read()
    #print(x)
    graph_def.ParseFromString(x)
    #tf.import_graph_def(graph_def, name='')

In [7]:
# 使用之前存下來的模型
X, Y_, prediction, accuracy, train_step, keep_prob, init_op= tf.import_graph_def(graph_def, name="", 
        return_elements=["X:0", "Y_:0", "prediction:0", 
                         "accuracy:0", "Adam", 'dropout/keep_prob:0', "init"])

In [ ]:
sess = tf.InteractiveSession()
init_op.run()
tf.summary.scalar(accuracy.op.name, accuracy)
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("log1", graph=sess.graph)

In [ ]:
for i in range(5000):
    rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
    if i%250 == 0:       
        summary_str, validation_accuracy = sess.run([summary_op, accuracy],
                        {X: validation_X[:1000], 
                                       Y_: validation_Y[:1000], 
                                       keep_prob: 1.0 })
        summary_writer.add_summary(summary_str, i)
        print("step %d, validation accuracy: %g"%(i, validation_accuracy))
    train_step.run({X: train_X[rnd_idx], Y_: train_Y[rnd_idx], keep_prob: 0.5 })

step 0, validation accuracy: 0.107
step 250, validation accuracy: 0.886
step 500, validation accuracy: 0.927
step 750, validation accuracy: 0.945
step 1000, validation accuracy: 0.951
step 1250, validation accuracy: 0.961
step 1500, validation accuracy: 0.969
step 1750, validation accuracy: 0.97


In [ ]:
summary_writer.close()
sess.close()

### 同時紀錄三種準確度

In [ ]:
sess = tf.InteractiveSession()
init_op.run()
acc_summary = tf.summary.scalar("accuracy", accuracy)
training_summary_writer = tf.summary.FileWriter("log2/training", graph=sess.graph)
validation_summary_writer = tf.summary.FileWriter("log2/validation", graph=sess.graph)
testing_summary_writer = tf.summary.FileWriter("log2/testing", graph=sess.graph)


In [ ]:
for i in range(5000):
    rnd_idx = np.random.choice(train_X.shape[0], 50, replace=False)
    if i%50 == 0:       
        summary_str, training_acc = sess.run([acc_summary, accuracy],
                        {X: train_X[:1000], Y_: train_Y[:1000], keep_prob: 1.0 })
        training_summary_writer.add_summary(summary_str, i)
        summary_str, validation_acc = sess.run([acc_summary, accuracy],
                        {X: validation_X[:1000], Y_: validation_Y[:1000], keep_prob: 1.0 })
        validation_summary_writer.add_summary(summary_str, i)
        summary_str, testing_acc = sess.run([acc_summary, accuracy],
                        {X: test_X[:1000], Y_: test_Y[:1000], keep_prob: 1.0 })
        testing_summary_writer.add_summary(summary_str, i)
        if i%250==0:
            print("step %d, train: %g, validation: %g, test: %g"%(i, training_acc, 
                                                              validation_acc, testing_acc))
    train_step.run({X: train_X[rnd_idx], Y_: train_Y[rnd_idx], keep_prob: 0.5 })

In [ ]:
testing_summary_writer.close()
validation_summary_writer.close()
training_summary_writer.close()
sess.close()